In [ ]:
# Importar las librerías necesarias
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF

import numpy as np
 
# Leer el dataset y crear el método de evaluación
reader = Reader()
ml_data = reader.read(fpath='datasets/ml-100k/u.data', fmt='UIRT', sep='\t')

eval_method = RatioSplit(
    data=ml_data,
    test_size=0.2,
    rating_threshold=0.0,
    exclude_unknowns=True,
    verbose=True,
    seed=42
)

rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


In [ ]:
# Definir modelos
model_mf = MF(k=10, max_iter=50, learning_rate=0.01)
model_pmf = PMF(k=10, max_iter=50, learning_rate=0.01)

# Entrenar modelos
model_mf.fit(eval_method.train_set)
model_pmf.fit(eval_method.train_set)

# Obtener historial de entrenamiento y lista de ítems. Esto sirve para no recomendar ítems que el usuario ya vió.

# Historial de ítems vistos por usuario
train_set = eval_method.train_set
user_rated = {}

user_indices, item_indices, _ = train_set.uir_tuple     # uir_tuple devuelve un array de tuplas (user_indices, item_indices, rating) para el conjunto de entrenamiento.

for uid, iid in zip(user_indices, item_indices):        # user_indices, item_indices son arrays paralelos → zip itera sobre ambos a la vez.
    if uid not in user_rated:
        user_rated[uid] = set()
    user_rated[uid].add(iid)


In [ ]:
# Generar recomendaciones Top-N por usuario
TOP_N = 5

# Usar los índices internos de Cornac para los ítems
all_items_internal_index = set(train_set.iid_map.values())

# Función para obtener las recomendaciones Top-N para un modelo dado
def get_top_n(model, name):
    top_n = {}
    for uid in train_set.uid_map.values():
        seen = user_rated.get(uid, set())
        candidates = all_items_internal_index - seen
        
        scores = [(iid, model.score(uid, iid)) for iid in candidates]
        scores.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [iid for iid, _ in scores[:TOP_N]]
    
    print(f"Top-{TOP_N} generado para modelo {name}")
    return top_n

# Generar listas
top_n_mf = get_top_n(model_mf, "MF")
top_n_pmf = get_top_n(model_pmf, "PMF")

Top-5 generado para modelo MF
Top-5 generado para modelo PMF


In [ ]:
def calcular_cobertura(top_n, all_items_set):
    # Unir todos los ítems recomendados por todos los usuarios
    items_recomendados = set()
    for recomendados in top_n.values():
        items_recomendados.update(recomendados)

    cobertura = len(items_recomendados) / len(all_items_set)
    return cobertura, len(items_recomendados)


cobertura_mf, cantidad_mf = calcular_cobertura(top_n_mf, all_items_internal_index)
cobertura_pmf, cantidad_pmf = calcular_cobertura(top_n_pmf, all_items_internal_index)

print(f"Total de ítems únicos en el dataset: {len(all_items_internal_index)}")
print(f"MF - Cobertura: {cobertura_mf:.4f} ({cantidad_mf} ítems únicos recomendados)")
print(f"PMF - Cobertura: {cobertura_pmf:.4f} ({cantidad_pmf} ítems únicos recomendados)")

**Resultados coverage**

| Modelo | Ítems únicos recomendados | Cobertura del catálogo |
| ------ | ------------------------- | ---------------------- |
| MF     | 323                       | 19.56%                 |
| PMF    | 532                       | 32.22%                 |
| Total  | 1651 ítems en el catálogo | 100%                   |


**Conclusiones**

**PMF cubre más del catálogo**

* PMF recomienda más ítems diferentes a lo largo de todos los usuarios.
* Su cobertura del 32% es significativamente más alta que la de MF (\~19%).
* Esto sugiere que PMF explora más y no se enfoca tanto en los mismos ítems populares.

**MF es más conservador**

* MF cubre sólo el 19% del catálogo.
* Recomienda menos ítems diferentes, lo cual puede deberse a que se concentra más en los ítems con mejor puntuación promedio.
* Esto lo hace más preciso, pero puede generar listas más repetitivas y reducir la novedad y exploración para el usuario.


**Trade-off entre cobertura y precisión**

Si recordamos los resultados anteriores de precisión (`Precision@5` y `NDCG@5`):

| Modelo | Precision\@5 | NDCG\@5    |
| ------ | ------------ | ---------- |
| MF     | 0.0732       | 0.0762     |
| PMF    | 0.0506       | 0.0492     |

* MF fue más preciso pero menos diverso.
* PMF fue menos preciso, pero más explorador del catálogo.

**Esto es el típico trade-off:**
Mayor precisión → menos cobertura.
Mayor cobertura → posible pérdida de precisión.

**Conclusión general**

| Modelo  | Fortalezas                              | Debilidades                                                   |
| ------- | --------------------------------------- | ------------------------------------------------------------- |
| MF  | Preciso, enfocado en relevancia         | Repite ítems, baja cobertura (riesgo de sesgo de popularidad) |
| PMF | Explora más el catálogo, mayor variedad | Menor precisión, puede recomendar ítems menos relevantes      |




In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

def calcular_diversidad(top_n, model, nombre):
    if nombre == "MF":
        item_factors = model.i_factors
    elif nombre == "PMF":
        item_factors = model.V
    else:
        raise AttributeError("No se encontraron vectores de ítems en el modelo.")
    diversidades = []

    for uid, recs in top_n.items():
        if len(recs) < 2:
            continue  # no se puede calcular diversidad con menos de 2 ítems

        # Calculamos la similitud coseno entre todos los pares de ítems
        pares = list(combinations(recs, 2))  # todas las combinaciones de 2 ítems
        sim_total = 0

        for i1, i2 in pares:
            vec1 = item_factors[i1]
            vec2 = item_factors[i2]
            sim = cosine_similarity([vec1], [vec2])[0][0]
            sim_total += sim

        promedio_sim = sim_total / len(pares)
        diversidad = 1 - promedio_sim
        diversidades.append(diversidad)

    diversidad_promedio = np.mean(diversidades)
    print(f"{nombre} - Diversidad intra-lista promedio: {diversidad_promedio:.4f}")
    return diversidad_promedio


div_mf = calcular_diversidad(top_n_mf, model_mf, "MF")
div_pmf = calcular_diversidad(top_n_pmf, model_pmf, "PMF")


**Resultados de diversidad intra-lista**

| Modelo | Diversidad intra-lista promedio |
| ------ | ------------------------------- |
| MF     |     0.6895                      |
| PMF    |     0.5866                      |

La diversidad intra-lista promedio mide cuán diferentes entre sí son los ítems que se le recomiendan a un mismo usuario:

* El valor está entre 0 y 1:

  * 0 = todos los ítems son casi iguales (lista homogénea).
  * 1 = todos los ítems son completamente distintos (lista muy variada).


**MF: 0.6895**

* Esto significa que las listas recomendadas por el modelo MF son razonablemente diversas.
* Dentro de cada Top-5, los ítems no son idénticos entre sí.
* El modelo logra buen balance entre precisión y variedad.

**PMF: 0.5866**

* Las listas de PMF son menos diversas.
* Aunque PMF tenía mejor cobertura del catálogo (recomendó más ítems distintos entre usuarios), dentro de cada lista individual tiende a agrupar ítems más parecidos.

**Conclusión general**

| Modelo  | Cobertura del catálogo | Diversidad intra-lista | Interpretación                                                    |
| ------- | ---------------------- | ---------------------- | ----------------------------------------------------------------- |
| MF  | 19.56%                 | 0.6895             | Menos ítems distintos en total, pero más variados por lista       |
| PMF | 32.22%             | 0.5866                 | Explora más el catálogo global, pero ofrece listas más homogéneas |

* MF prioriza precisión y ofrece listas más balanceadas internamente.
* PMF explora más ítems distintos (mejor cobertura), pero cada lista tiene menos variedad.

**Si tu objetivo es:**

* Satisfacción del usuario a largo plazo → preferís más diversidad (MF).
* Descubrimiento global de contenido (catálogo) → preferís más cobertura (PMF).